<div >
<img src = "../../banner.jpg" />
</div>

# CARTs, Bagging and Random Forests

## Predicting House Prices

Our objective today is to construct a model to predict house prices. From Rosen's landmark paper "Hedonic Prices and Implicit Markets: Product Differentiation in Pure Competition" (1974), we know that a vector of its characteristics describes a differentiated good.

In the case of a house, these characteristics may include structural attributes (e.g., number of bedrooms), neighborhood public services (e.g., local school quality), and local amenities (e.g., crime, air quality, etc). Thus, we can write the market price of the house as:

$$
Price=f(structural\,attributes,amenities,...)
$$


However, Rosen's theory doesn't tell us much about the functional form of $f$. 

## Example Problem Set

In [1]:
#packages
require("pacman")
p_load("tidyverse","ggplot2")

Loading required package: pacman



Data on Kaggle: https://www.kaggle.com/competitions/uniandes-bdml-202320-ps2/overview

In [3]:
test<- read.csv("test.csv")


train<-read.csv("train.csv")

head(test)

,property_id,city,price,month,year,surface_total,surface_covered,rooms,bedrooms,bathrooms,property_type,operation_type,lat,lon,title,description
,<chr>,<chr>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,d56159653f5d2abc5ed5acd7,Bogotá D.C,NA,10,2020,NA,NA,3,3,NA,Apartamento,Venta,4.666575,-74.04877,apartamento la cabrera,este apartamento tiene unos hermosos acabados espacios amplios y generosos para disfrutar en familia y con amigos su amplia zona social la cocina muy funcional para quienes disfrutan preparando cenas especiales cuenta con todo lo necesario y excelente iluminacion al fondo la zona de ropas y un cuarto util que el propietario entrega con repisas para almacenamiento tiene un espacio que actualmente se destino a estar de tv puede ser tambien utilizado para un comodo estudio para trabajo en casa la alcoba principal con vestier y bano privado tiene una excelente iluminacion y es muy amplia cuenta con dos alcobas mas cada una con closet comparten un bano adicionalmente esta el bano para la zona social el parqueadero permite acomodar dos vehiculos en linea esta apartamento es especial por su excelente diseno para quienes disfrutan del confort esta en una zona exclusiva segura y tranquila que ademas goza de alta izacion
2,45550aa81c0151f1fd9fd8fe,Bogotá D.C,NA,10,2020,NA,NA,4,4,NA,Apartamento,Venta,4.672834,-74.05538,apartamento en parque virrey,acogedor apartamento frente al parque el virrey costado sur ideal para familias cuenta con 3 habitaciones amplias habitacion principal con bano y walking closet cuarto de servicio con su respectivo bano cuenta con un area de 135 m2 balcon terraza interior de uso exclusivo de 11 m2 sala comedor con chimenea estudio independiente cocina cerrada con opcion de abrirla 2 parqueaderos en linea un deposito interno y otro externo en el 9 piso el edificio cuenta con caldera planta electrica
3,735deef3382aa15930482f3e,Bogotá D.C,NA,10,2020,NA,NA,2,2,NA,Apartamento,Venta,4.675888,-74.05444,apartamento en exclusivo sector el virrey,quisieras vivir en un sector exclusivo de bogota muy tranquilo y seguro te traemos este apartamento en el barrio virrey a dos cuadras del parque virrey cerca al cc andino y parque de la 93 con solo 2 anos de construido acabados aaa cocina abierta tipo americana puerta de seguridad exterior con balcon y un muy buen de administracion para el sector el edificio cuenta con caldera planta electrica 2 ascensores terraza bbq cuarto de escoltas gimnasio parqueadero de visitantes
4,0dd49fa8bb7be52b851c6dc9,Bogotá D.C,NA,10,2020,NA,NA,3,3,NA,Apartamento,Venta,4.683480,-74.05243,hermoso apartamento con terraza en chico,lindo apartamento con ascensor privado con un area de 1463 mts y una increible terraza de 1129 mts distribuido con sala comedor independiente balcon bano auxiliar 3 alcobas la principal muy comoda con vestier y bano con tina bano de alcobas cocina remodelada con zona de ropas cuarto y bano de servicio 2 garajes independientes y deposito el conjunto cuenta con teatrino salon de juegos para ninos parque salon social gimnasio y sala de internet
5,7d3220a91e78f8bc2924d1cd,Bogotá D.C,NA,10,2020,NA,NA,3,3,NA,Apartamento,Venta,4.669054,-74.05105,espectacular penthouse duplex en cabrera,presentamos este increible penthouse duplex en cabrera uno de los barrios mas exclusivos de la ciudad con altas expectativas de izacion dadas las bondades de sus viviendas y lo estrategico de su ubicacion cerca de importantes zonas financieras y centros comerciales tiene un area de 234 mts es un piso sexto interior pero bastante iluminado cuenta con 3 habitaciones 3 banos dos estudios dos chimeneas dos balcones a los que se accede por la zona social cuarto y bano de servicio dos garajes en linea y deposito
6,d3054072b2175deec2c1f8dc,Bogotá D.C,NA,10,2020,NA,NA,NA,1,2,Apartamento,Venta,4.646160,-74.06468,apartamento en bogota chapinero,apartaestudio en tipo loft exterior para estrenar con na acustica de reduccion al ruido ubicado estrategicamen

In [4]:
test<-test  %>% mutate(sample="test")
train<-train  %>% mutate(sample="train")

db_ps<-rbind(test,train)
table(db_ps$sample)


 test train 
10286 38644 

In [5]:
p_load("sf")
db_ps <- st_as_sf(
  db_ps,
  # "coords" is in x/y order -- so longitude goes first!
  coords = c("lon", "lat"),
  # Set our coordinate reference system to EPSG:4326,
  # the standard WGS84 geodetic coordinate reference system
  crs = 4326
)

In [6]:
p_load("leaflet")

pal <- colorFactor(
  palette = c('red', 'green'),
  domain = db_ps$sample
)

map<-leaflet() %>% 
  addTiles() %>%  #capa base
  addCircles(data=db_ps,col=~pal(sample)) #capa casas
map 

HTML widgets cannot be represented in plain text (need html)

In [7]:
#Extra steps bc of jupyter notebook (if you are in RStudio, you don't need these steps)
p_load("htmlwidgets") 
p_load("IRdisplay")
saveWidget(map, file="m.html")
display_html('<iframe width="900" height="700" src="m.html"></iframe>')

### Distance to CBD?


<div >
<img width="400" height="100" src = "../figures/precio_suelo.png" />
</div>

In [8]:
p_load("tmaptools") #needs to install p_load("geojsonio")
cbd <- geocode_OSM("Centro Internacional, Bogota", as.sf=T) 
cbd

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



,query,lat,lon,lat_min,lat_max,lon_min,lon_max,bbox,point
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<POLYGON [°]>,<POINT [°]>
1,"Centro Internacional, Bogota",4.613615,-74.07057,4.613373,4.613889,-74.07098,-74.07023,POLYGON ((-74.07098 4.61337...,POINT (-74.07057 4.613615)


In [9]:
db_ps$DCBD<-st_distance(x = db_ps, y = cbd)

In [10]:
head(db_ps$DCBD)

Units: [m]
         [,1]
[1,] 6365.057
[2,] 6796.677
[3,] 7151.506
[4,] 8024.584
[5,] 6533.207
[6,] 3677.279

In [11]:
db_ps  %>% st_drop_geometry() %>% group_by(sample)  %>% summarize(mean(DCBD))

sample,mean(DCBD)
<chr>,<[m]>
test,6256.975 [m]
train,9729.774 [m]


In [12]:
train_data<-db_ps  %>% filter(sample=="train")  %>% select(price,DCBD)  %>% na.omit()

In [13]:
p_load("caret")

fitControl<-trainControl(method ="cv",
                         number=5)

set.seed(123)
tree <- train(
    log(price) ~    DCBD,
    data=train_data,
    method = "rpart",
    trControl = fitControl,
    tuneLength=100
)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


In [14]:
tree

CART 

38644 samples
    2 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 30916, 30916, 30914, 30916, 30914 
Resampling results across tuning parameters:

  cp            RMSE       Rsquared    MAE      
  0.0002736310  0.4028070  0.15469660  0.3230243
  0.0002763787  0.4027600  0.15470117  0.3230494
  0.0002764837  0.4027600  0.15470117  0.3230494
  0.0002776611  0.4027491  0.15467758  0.3230566
  0.0002777680  0.4027491  0.15467758  0.3230566
  0.0002803098  0.4027764  0.15441543  0.3232025
  0.0002824160  0.4028524  0.15407408  0.3233347
  0.0002826423  0.4028383  0.15411837  0.3233248
  0.0002871725  0.4029698  0.15341054  0.3235066
  0.0002875731  0.4029938  0.15330614  0.3235308
  0.0002883653  0.4029754  0.15336055  0.3235299
  0.0002883810  0.4029754  0.15336055  0.3235299
  0.0002894745  0.4029692  0.15334561  0.3235434
  0.0002926435  0.4030686  0.15284313  0.3237782
  0.0002934941  0.4030781  0.15276013  0.3238423
  0.0002940498  

In [15]:
test_data<-db_ps  %>% filter(sample=="test")  
test_data$pred_tree<-predict(tree,test_data)

In [16]:
head(test_data  %>% select(property_id,pred_tree))

,property_id,pred_tree,geometry
,<chr>,<dbl>,<POINT [°]>
1,d56159653f5d2abc5ed5acd7,20.09564,POINT (-74.04877 4.666575)
2,45550aa81c0151f1fd9fd8fe,20.05442,POINT (-74.05538 4.672834)
3,735deef3382aa15930482f3e,20.15780,POINT (-74.05444 4.675888)
4,0dd49fa8bb7be52b851c6dc9,20.25569,POINT (-74.05243 4.68348)
5,7d3220a91e78f8bc2924d1cd,20.09564,POINT (-74.05105 4.669054)
6,d3054072b2175deec2c1f8dc,19.99960,POINT (-74.06468 4.64616)


In [17]:
test_data <- test_data   %>% st_drop_geometry()  %>% mutate(pred_tree=exp(pred_tree))
head(test_data  %>% select(property_id,pred_tree), n=100)

,property_id,pred_tree
,<chr>,<dbl>
1,d56159653f5d2abc5ed5acd7,533859379
2,45550aa81c0151f1fd9fd8fe,512297915
3,735deef3382aa15930482f3e,568098099
4,0dd49fa8bb7be52b851c6dc9,626519694
5,7d3220a91e78f8bc2924d1cd,533859379
6,d3054072b2175deec2c1f8dc,484972573
7,b7574b06d593fca4eac59815,893395420
8,79e822115c5c1a4531366001,584845434
9,d3ed5316a463d38c1bd321d0,557405445


In [18]:
submit<-test_data  %>% select(property_id,pred_tree)
submit <- submit  %>% rename(price=pred_tree)
write.csv(submit,"tree.csv",row.names=FALSE)